### Getting started on ANN with MLFlow

Quickstart : Compare runs, choose a model, and deploy it to a REST API

In this project : 

-> Run a hyper parameter sweep on a training script <br>
-> Compare the results of the runs in the mlflow ui <br>
-> Choose the best run and register it as a model <br>
-> Deploy the model using REST API <br>
-> Buiild a container image suitable for deployment to a cloud platform.



In [1]:
import pandas as pd 
# hyper opt is the library that helps us do hyper parameter tuning in ANNs
import numpy as np 
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow 
from mlflow.models import infer_signature

In [2]:
import keras

In [3]:
### load the dataset 

data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep = ";",
)
data

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.0              0.27         0.36            20.7      0.045   
1               6.3              0.30         0.34             1.6      0.049   
2               8.1              0.28         0.40             6.9      0.050   
3               7.2              0.23         0.32             8.5      0.058   
4               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4893            6.2              0.21         0.29             1.6      0.039   
4894            6.6              0.32         0.36             8.0      0.047   
4895            6.5              0.24         0.19             1.2      0.041   
4896            5.5              0.29         0.30             1.1      0.022   
4897            6.0              0.21         0.38             0.8      0.020   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                    45.0                 170.0  1.00100  3.00       0.45   
1                    14.0                 132.0  0.99400  3.30       0.49   
2                    30.0                  97.0  0.99510  3.26       0.44   
3                    47.0                 186.0  0.99560  3.19       0.40   
4                    47.0                 186.0  0.99560  3.19       0.40   
...                   ...                   ...      ...   ...        ...   
4893                 24.0                  92.0  0.99114  3.27       0.50   
4894                 57.0                 168.0  0.99490  3.15       0.46   
4895                 30.0                 111.0  0.99254  2.99       0.46   
4896                 20.0                 110.0  0.98869  3.34       0.38   
4897                 22.0                  98.0  0.98941  3.26       0.32   

      alcohol  quality  
0         8.8        6  
1         9.5        6  
2        10.1        6  
3         9.9        6  
4         9.9        6  
...       ...      ...  
4893     11.2        6  
4894      9.6        5  
4895      9.4        6  
4896     12.8        7  
4897     11.8        6  

[4898 rows x 12 columns]

In [4]:
data.isnull().any() # checking for any null values in the data 

fixed acidity           False
volatile acidity        False
citric acid             False
residual sugar          False
chlorides               False
free sulfur dioxide     False
total sulfur dioxide    False
density                 False
pH                      False
sulphates               False
alcohol                 False
quality                 False
dtype: bool

In [5]:
# quality is what that has to be predicted 

# splitting the data into training ,validation and test set 

train,test = train_test_split(data, test_size=0.25,random_state = 42)
train

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
2835            6.3              0.25         0.22            3.30      0.048   
1157            7.8              0.30         0.29           16.85      0.054   
744             7.4              0.38         0.27            7.50      0.041   
1448            7.4              0.16         0.49            1.20      0.055   
3338            7.2              0.27         0.28           15.20      0.046   
...             ...               ...          ...             ...        ...   
4426            6.2              0.21         0.52            6.50      0.047   
466             7.0              0.14         0.32            9.00      0.039   
3092            7.6              0.27         0.52            3.20      0.043   
3772            6.3              0.24         0.29           13.70      0.035   
860             8.1              0.27         0.35            1.70      0.030   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
2835                 41.0                 161.0  0.99256  3.16       0.50   
1157                 23.0                 135.0  0.99980  3.16       0.38   
744                  24.0                 160.0  0.99535  3.17       0.43   
1448                 18.0                 150.0  0.99170  3.23       0.47   
3338                  6.0                  41.0  0.99665  3.17       0.39   
...                   ...                   ...      ...   ...        ...   
4426                 28.0                 123.0  0.99418  3.22       0.49   
466                  54.0                 141.0  0.99560  3.22       0.43   
3092                 28.0                 152.0  0.99129  3.02       0.53   
3772                 53.0                 134.0  0.99567  3.17       0.38   
860                  38.0                 103.0  0.99255  3.22       0.63   

      alcohol  quality  
2835     10.5        6  
1157      9.0        6  
744      10.0        5  
1448     11.2        6  
3338     10.9        6  
...       ...      ...  
4426      9.9        6  
466       9.4        6  
3092     11.4        6  
3772     10.6        6  
860      10.4        8  

[3673 rows x 12 columns]

In [6]:
train_x = train.drop(['quality'],axis=1).values # .values is used to convert the values into an array 
train_y = train[['quality']].values.ravel() # .ravel() converts the data into a single dim array 

## test dataset 

test_x = test.drop(['quality'],axis=1).values
test_y = test[['quality']].values.ravel() 

# validation dataset - splitting train data 

train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20, random_state = 42)

signature = infer_signature(train_x,train_y)


In [16]:
### ANN model 

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    # define the model architecture 
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)
    
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )
    
    ## compile the model 
    
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate = params["lr"],momentum = params["momentum"]),
                  loss="mean_squared_error",
                  metrics = [keras.metrics.RootMeanSquaredError()])
    
    ## train the ANN model with lr and momentum params 
    
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs = epochs,
                  batch_size = 64)
        ## evaluate the model 
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)
        eval_rmse = eval_result[1]
        
        ## log the params and metrics 
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)
        
        ## log the model 
        mlflow.tensorflow.log_model(model,"model1",signature=signature)
        return {"loss": eval_rmse, "status": STATUS_OK, "model":model}
    
    
    

In [17]:
def objective(params):
    # MLFlow will track the parameters and results for each run 
    result = train_model(
        params,
        epochs=3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y
    )
    
    return result




In [18]:
## set all the parameters 

space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)), # meanss the learning rate values vary from 10^-5 to 10^-1
    "momentum" : hp.uniform("momentum",0.0,1.0)
}

In [19]:
mlflow.set_experiment("/wine-quality") # setting the experiment 
with mlflow.start_run():
    # conduct the hyper parameter search using hyper opt 
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals = 4,
        trials = trials
    )
    
    # fetch the details of the best run 
    best_run = sorted(trials.results, key=lambda x:x["loss"])[0]
    

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 36s 819ms/step - loss: 40.1150 - root_mean_squared_error: 6.3336
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40.6279 - root_mean_squared_error: 6.3740   
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 40.5025 - root_mean_squared_error: 6.3642 - val_loss: 40.4077 - val_root_mean_squared_error: 6.3567

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 37.0523 - root_mean_squared_error: 6.0871
39/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 39.0077 - root_mean_squared_error: 6.2455 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38.8168 - root_mean_squared_error: 6.2303 - val_loss: 38.7428 - val_root_mean_squared_error: 6.2244

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 39.2997 - root_mean_squared_error: 6.2689
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 38.1487 - root_mean_squared_error: 6.

2025/10/21 22:08:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 21s 481ms/step - loss: 44.3858 - root_mean_squared_error: 6.6623
41/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 37.1803 - root_mean_squared_error: 6.0885   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 29.7742 - root_mean_squared_error: 5.4566 - val_loss: 18.3262 - val_root_mean_squared_error: 4.2809

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 19.9541 - root_mean_squared_error: 4.4670
40/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15.5944 - root_mean_squared_error: 3.9428 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.4675 - root_mean_squared_error: 3.5309 - val_loss: 7.6800 - val_root_mean_squared_error: 2.7713

Epoch 3/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 7.5269 - root_mean_squared_error: 2.7435
37/46 ━━━━━━

2025/10/21 22:09:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 19s 433ms/step - loss: 33.4975 - root_mean_squared_error: 5.7877
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - root_mean_squared_error: nan          
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: nan - root_mean_squared_error: nan
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - root_mean_squared_error: nan 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan

Epoch 3/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: nan - root_mean_squared_error: nan
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - root_m

2025/10/21 22:09:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 21s 479ms/step - loss: 28.9192 - root_mean_squared_error: 5.3777
40/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.1216 - root_mean_squared_error: 3.0920   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.6239 - root_mean_squared_error: 2.1503 - val_loss: 0.6973 - val_root_mean_squared_error: 0.8350

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.5120 - root_mean_squared_error: 0.7156
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6467 - root_mean_squared_error: 0.8038 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6168 - root_mean_squared_error: 0.7854 - val_loss: 0.5666 - val_root_mean_squared_error: 0.7527

Epoch 3/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.4937 - root_mean_squared_error: 0.7026
41/46 ━━━━━━━━━━━

2025/10/21 22:10:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [01:20<00:00, 20.08s/trial, best loss: 0.7323207259178162]
